## 원티드 - 데이터 엔지니어
* 데이터 엔지니어(job-card) -> 세부 페이지 들어가서 JD 스크래핑

### 데이터 엔지니어(job-card) 데이터 가져오기

In [1]:
# 원티드 - 데이터 엔지니어 > 응답률순 정렬
url = 'https://www.wanted.co.kr/wdlist/518/655?country=kr&job_sort=company.response_rate_order&years=-1&locations=all'

In [2]:
import pandas as pd

In [17]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options)

driver.implicitly_wait(5)
driver.get(url)

last_height = driver.execute_script('return document.body.scrollHeight')
while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(2)
    new_height = driver.execute_script('return document.body.scrollHeight')
    if new_height == last_height:
        break
    last_height = new_height

###### 원티트 페이지
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    container = soup.select('#__next > div.JobList_cn__t_THp > div > div > div.List_List_container__JnQMS > ul')
#print(container[0].prettify())

job_card = container[0].find_all('div', {'class': 'Card_className__u5rsb'})
#print(job_card)

In [18]:
driver.quit()  ### 이걸 집어넣어야해!!!안그면 오류난다!!!!!!!

In [19]:
######## 뷰티풀숲!!!! 이거 자꾸 까먹는다아아아아아 ###############

print(job_card[0].find('a')['href'])
print(job_card[0].find('a')['data-company-name'])
print(job_card[0].find('a')['data-position-name'])

/wd/132626
악어디지털
딥러닝모델개발자


In [21]:
print(len(job_card))

159


In [22]:
## 데이터 엔지니어 관련 공고 가져오기( 회사명 - 직무명 -페이지주소 )
job_lists = []
for i in range(len(job_card)):
    address = job_card[i].find('a')['href']
    company_name = job_card[i].find('a')['data-company-name']
    position = job_card[i].find('a')['data-position-name']
    
    ## 근데 페이지 주소는 '/wd/1234' 이런식이니까 'www.wanted.co.kr'을 앞에 넣어주면
    url2 = 'https://www.wanted.co.kr' + address
    
    job_lists.append([company_name, position, url2])

#print(job_lists)

In [32]:
## 위의 결과를 데이터프레임으로 만들어보자
df1 = pd.DataFrame(job_lists, columns = ['company', 'job','url'])
df1
df1.to_csv('company.csv')

## 공고 세부 페이지 들어가서 JD 가져오기

** 근데 문제는 우대사항이 없는 회사가 있어서 나중에 데이터 다 뽑히면 정리해야함
** 어떤 곳은 우대사항에 회사복지가 들어간 곳이 있음......

In [24]:
job_urls = df1['url']
job_urls

0      https://www.wanted.co.kr/wd/132626
1      https://www.wanted.co.kr/wd/103169
2      https://www.wanted.co.kr/wd/130687
3      https://www.wanted.co.kr/wd/126811
4      https://www.wanted.co.kr/wd/129440
                      ...                
154     https://www.wanted.co.kr/wd/74156
155     https://www.wanted.co.kr/wd/53517
156     https://www.wanted.co.kr/wd/74154
157     https://www.wanted.co.kr/wd/74155
158     https://www.wanted.co.kr/wd/74151
Name: url, Length: 159, dtype: object

## 최종적으로 정리하면 
* 일단 뽑아놓은 채용 공고 정리 페이지 주소를 하나씩 가져와서 
* 회사명, 주요업무, 자격요건, 우대사항 정리하기

In [33]:
############### 종합적으로 정리하면 
results = []

###### 잘 되는지 일단 뽑아온 주소 5개만 해보자!! 
for i in range(0,3):

## 모든 주소 가져오려면 이걸 써야해!!!
# for i in range(len(job_urls)):

    url2 = job_urls[i] 
    
    ## 셀레니엄
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options)

    driver.implicitly_wait(0.1)
    driver.get(url2)
    
    #### 채용공고에 해당되는 박스의 클래스 가져오고 
    jd_box = driver.find_element(By.CLASS_NAME,'JobContent_descriptionWrapper__SM4UD' )

    ## 작성내용은 태그 p로 되어 있으니까 
    contents = jd_box.find_elements(By.TAG_NAME, 'p')
        
    ## 해당 내용들을 정리하면
    company_pr = contents[0]    # 회사명
    to_do = contents[1]    #주요업무
    need_to = contents[2]   #자격요건
    prefer = contents[3]   #우대사항 
    
    ## 회사소개
    value = company_pr.find_element(By.TAG_NAME, 'span')
    content_company_pr = value.get_attribute('innerHTML')

    ## 주요업무
    value = to_do.find_element(By.TAG_NAME, 'span')
    content_to_do = value.get_attribute('innerHTML')

    ## 자격요건
    value = need_to.find_element(By.TAG_NAME, 'span')
    content_need_to = value.get_attribute('innerHTML')

    ## 우대사항
    value = prefer.find_element(By.TAG_NAME, 'span')
    content_prefer = value.get_attribute('innerHTML')

    ## 전부다 불러와서 행으로 들어갈 데이터 만들어 
    results.append([content_company_pr,content_to_do,content_need_to, content_prefer, url2])
    
    ## 닫어!!!!!!!!!
    driver.quit()

## 데이터 프레임 만들기    
df2 = pd.DataFrame(results, columns =['company', 'to_do', 'need_to', 'preference', 'url'])
df2

,company,to_do,need_to,preference,url
0,[회사소개] <br><br>악어디지털은 지난 2014년 문서 전자화 O2O 서비스 ...,• 데이터분석<br>• 딥러닝 모델 훈련 및 모니터링<br>• 데이터 파이프라인 시...,• 2년 이상 경력자<br>• python을 이용한 프로그램 작성 경험<br>• 딥...,• OCR 데이터 활용 경험<br>• 스캔된 문서 처리 경험,https://www.wanted.co.kr/wd/132626
1,[수성엔지니어링 기술연구소 개발팀 소개]<br>수성엔지니어링은 한국 엔지니어링 업계...,• Python 기반으로 공간데이터 처리 알고리즘 개발 (Docker/AWS ECS...,• Python을 능숙하게 다룰 수 있는 개발자<br>• 객체지향 프로그래밍에 대한...,"• Slack , Notion , GitHub등 협업툴 사용에 익숙하신 분<br>•...",https://www.wanted.co.kr/wd/103169
2,"IT 비즈니스 혁신을 선도하는 AI 스타트업, 아인시스에이아이랩(이하 에이아이랩) ...",• 데이터 분석을 위한 알고리즘 및 개발 방향과 범위를 산정해요<br>• 데이터 수...,• 이공계 졸업(예정자) 혹은 관련 경력을 보유하신 분<br>• 네트워크 애플리케이...,• 스타트업 업무경험이 있으신 분<br>• Rust 개발 경험이 있으신 분<br>•...,https://www.wanted.co.kr/wd/130687


In [31]:
df2.to_csv('de.csv')